# Title by Owner

## Imports

In [20]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datetime import datetime
from os import path
from utils.callbacks import SaveBestModelInMemory

## Parameters

In [18]:
BATCH_SIZE = 64 # Number of samples in a mini batch
EPOCHS = 10 # Number of training epochs before the training is stopped
NUM_CLASSES = 12
RANDOM_STATE = 42 # Seed for rng to make everything reproducible and deterministic af
TEST_SPLIT = 0.15 # Percent of data to use for validation/testing
TENSORBOARD_LOGS_PATH = "tensorboard-logs"
SAVED_MODELS_PATH = "saved-models"

## Data Loading

In [12]:
data = np.load(file="../dataset/x_train.npy")
labels = tf.keras.utils.to_categorical(np.load(file="../dataset/y_train.npy"), num_classes=NUM_CLASSES)

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

# Make sure everything was loaded correctly:
print(f"All samples shape: {data.shape}, all labels shape: {labels.shape}")
print(f"Train samples shape: {train_data.shape}, Train labels shape: {train_labels.shape}")
print(f"Test samples shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

All samples shape: (2429, 36, 6), all labels shape: (2429, 12)
Train samples shape: (2064, 36, 6), Train labels shape: (2064, 12)
Test samples shape: (365, 36, 6), Test labels shape: (365, 12)


## Preprocessing

In [13]:
# Define preprocessing pipeline/code

## Model Definition

In [14]:
# Create the model inside the function this is a dummy implementation
def build_model(name: str, input_shape: tuple[int,...], classes: int) -> tf.keras.Model:
    # Build the neural network layer by layer
    input_layer = tf.keras.layers.Input(shape=input_shape, name="Input")

    # Feature extractor
    lstm = tf.keras.layers.LSTM(128, return_sequences=True)(input_layer)
    lstm = tf.keras.layers.LSTM(128)(lstm)
    dropout = tf.keras.layers.Dropout(.5, seed=RANDOM_STATE)(lstm)

    # Classifier
    classifier = tf.keras.layers.Dense(128, activation="relu")(dropout)
    output_layer = tf.keras.layers.Dense(classes, activation="softmax")(classifier)

    # Connect input and output through the Model class
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name=name)

    # Compile the model
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics="accuracy")

    # Return the model
    return model

## Training

In [21]:
input_shape = train_data.shape[1:]
classes = NUM_CLASSES
model_name = "My-awesome-model" # Give your model an awesome name for a 2% percent accuracy increase.

model = build_model(model_name, input_shape, classes)
model.summary()

run_id = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
current_tensorboard_log_dir = f"{TENSORBOARD_LOGS_PATH}/{model_name}/{run_id}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=current_tensorboard_log_dir)
print(f"Run tensorboard in a separate process with:\n"
      f"tensorboard --logdir {path.abspath(TENSORBOARD_LOGS_PATH)}\nor\n"
      f"tensorboard --logdir {path.abspath(current_tensorboard_log_dir)}")

best_weights_callback = SaveBestModelInMemory(metric="val_loss")

model.fit(x=train_data, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(test_data, test_labels), callbacks=[tensorboard_callback, best_weights_callback])

Model: "My-awesome-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 36, 6)]           0         
                                                                 
 lstm_4 (LSTM)               (None, 36, 128)           69120     
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 12)                1548      
                                                                 
Total params: 218,764
Trainable params: 218,764
No

## Optional: Save model in memory

In [22]:
model.set_weights(best_weights_callback.best_weights)
model.save(f"{SAVED_MODELS_PATH}/{model_name}/{run_id}")

INFO:tensorflow:Assets written to: saved-models/My-awesome-model/2022-12-12-16-32-29/assets


INFO:tensorflow:Assets written to: saved-models/My-awesome-model/2022-12-12-16-32-29/assets
